# Arrayed SD

Arrayed SD allows for arrays to be used in SD models. 

First let's look at how arrayed SD works. We start with some boilerplate to get a BPTK project up and running:

In [1]:
from BPTK_Py import Model
import BPTK_Py

model = Model(starttime=0.0, stoptime=20.0, dt=1.0, name="Test Model")

Next we can start creating arrayed components. There are two options for arrayed elements: Vectors (one dimensional arrays) and Matrices (two dimensional arrays). These can be declared as follows:

In [2]:
vector = model.converter("converter_vector")

# create a vector of length 2 with values 2.0, 3.0
vector.setup_vector(2, [2.0, 3.0])

matrix = model.converter("converter_matrix")

# create a matrix of size 2x3 with values [2.0, 3.0, 4.0], [5.0, 6.0, 7.0]
matrix.setup_matrix([2, 3], [[2.0, 3.0, 4.0], [5.0, 6.0, 7.0]])

A variety of operations can be performed on vectors and matrices. Below are some examples, for a more detailed list see the documentation.

In [3]:
# multiply the vector with the matrix
multiplied = model.converter("multiplied")
multiplied.equation = vector.dot(matrix)

# get the second biggest element of the vector
rank = model.converter("rank")
rank.equation = matrix.arr_rank(2)

# Let's get the output of the rank converter
data = rank.plot(return_df=True)
data[rank.name][1]

6.0

## Named Arrays
Besides vectors and matrices named arrays are possible. These arrays can make code more readable but do not allow for some functions to be used which require indexed matrices (e.g. the dot operator). Below is an example.

In [48]:
named = model.converter("named")
named["tax_germany"] = 0.19
named["tax_switzerland"] = 0.08
named["tax_austria"] = 0.20

# todo: this does not work yet
data = named.plot(return_df=True)
#data[named.name]["tax_germany"]

print(data[named.name].iloc[0].columns[0], data[named.name].iloc[0].values[0][0])
print(data[named.name].iloc[1].columns[0], data[named.name].iloc[1].values[0][0])
print(data[named.name].iloc[2].columns[0], data[named.name].iloc[2].values[0][0])

named[tax_austria] 0.2
named[tax_germany] 0.19
named[tax_switzerland] 0.08


## Plotting
Plotting vectors and matrices is straightforward. Let's plot the vector below:

In [5]:
scenario_manager = {"sm": {"model": model}}

bptk = BPTK_Py.bptk()
bptk.register_scenario_manager(scenario_manager)
bptk.register_scenarios(
    scenario_manager="sm",
    scenarios=
    {
        "testScenario":{}
    }
)

In [6]:
from BPTK_Py.visualizations import SimpleDashboard
import ipywidgets as widgets

dashboard = SimpleDashboard(bptk, scenario_manager="sm", scenario="testScenario")

plot = dashboard.add_plot(
    equations=["converter_vector[0]", "converter_vector[1]"], 
    names=["VectorX", "VectorY"],
    title="Vector",
    x_label="X",
    y_label="Y",
)

display(plot)
dashboard.start()

Output()

Similarly matrices can be plotted.

In [7]:
from BPTK_Py.visualizations import SimpleDashboard
import ipywidgets as widgets

dashboard = SimpleDashboard(bptk, scenario_manager="sm", scenario="testScenario")

plot = dashboard.add_plot(
    equations=["converter_matrix[0][0]", "converter_matrix[0][1]", "converter_matrix[0][2]", "converter_matrix[1][0]", "converter_matrix[1][1]", "converter_matrix[1][2]"],
    names=["Matrix00", "Matrix01", "Matrix02", "Matrix10", "Matrix11", "Matrix12"],
    title="Matrix",
    x_label="X",
    y_label="Y",
)

display(plot)
dashboard.start()

Output()

The same works for named arrays:

In [8]:
from BPTK_Py.visualizations import SimpleDashboard
import ipywidgets as widgets

dashboard = SimpleDashboard(bptk, scenario_manager="sm", scenario="testScenario")

plot = dashboard.add_plot(
    equations=["named[tax_germany]", "named[tax_switzerland]", "named[tax_austria]"],
    names=["Germany", "Switzerland", "Austria"],
    title="Taxes",
    x_label="X",
    y_label="Y",
)

display(plot)
dashboard.start()

Output()

## Function documentation

Below is documentation of all relevant arrayed SD functions. First, let's declare some basic elements to do operations on:

In [9]:
constant = model.converter("converter_constant")
constant.equation = 100.0

vector3 = model.converter("converter_vector3")
vector3.setup_vector(3, [2.0, 3.0, 4.0])

matrix33 = model.converter("converter_matrix3")
matrix33.setup_matrix([3, 3], [[2.0, 3.0, 4.0], [5.0, 6.0, 7.0], [8.0, 9.0, 10.0]])

matrix32 = model.converter("converter_matrix32")
matrix32.setup_matrix([3, 2], [[2.0, 3.0], [5.0, 6.0], [8.0, 9.0]])

### Array Sum
Calculates the element-wise sum of an array.

Parameter:
    - include_all: Boolean -> If true, includes both numbered and named elements.

Example (2.0 + 3.0 + 4.0 = 9.0):

In [10]:
arr_sum_converter = model.converter("arr_sum_converter")
arr_sum_converter.equation = vector3.arr_sum()

data = arr_sum_converter.plot(return_df=True)
assert(data[arr_sum_converter.name][0] == 9.0)
data[arr_sum_converter.name][0]

9.0

### Array Product
Calculates the element-wise product of an array.

Parameter:
    - include_all: Boolean -> If true, includes both numbered and named elements.

Example (2.0 * 3.0 * 4.0 = 24.0):

In [11]:
arr_prod_converter = model.converter("arr_prod_converter")
arr_prod_converter.equation = vector3.arr_prod()

data = arr_prod_converter.plot(return_df=True)
assert(data[arr_prod_converter.name][0] == 24.0)
data[arr_prod_converter.name][0]

24.0

### Array rank
Calculates the nth highest element of an array.

Parameter: 
    - rank: int -> The rankth highest element will be returned.

Example: (third highest of matrix33 = 8.0):

In [12]:
arr_rank_converter = model.converter("arr_rank_converter")
arr_rank_converter.equation = matrix33.arr_rank(3)

data = arr_rank_converter.plot(return_df=True)
assert(data[arr_rank_converter.name][0] == 8.0)
data[arr_rank_converter.name][0]

8.0

### Array Mean
Calculates the element-wise mean of an array.

Example (mean of matrix32 = 5.5):

In [13]:
arr_mean_converter = model.converter("arr_mean_converter")
arr_mean_converter.equation = matrix32.arr_mean()

data = arr_mean_converter.plot(return_df=True)
assert(data[arr_mean_converter.name][0] == 5.5)
data[arr_mean_converter.name][0]

5.5

### Array Median
Calculates the element-wise median of an array.

Example (meadian of matrix32 = 5.5):

In [14]:
arr_median_converter = model.converter("arr_median_converter")
arr_median_converter.equation = matrix32.arr_median()

data = arr_median_converter.plot(return_df=True)
assert(data[arr_median_converter.name][0] == 5.5)
data[arr_median_converter.name][0]

5.5

### Array Standard Deviation
Calculates the element-wise standard deviation of an array.

Example (standard deviation of matrix32 = 2.5):

In [15]:
arr_stddev_converter = model.converter("arr_stddev_converter")
arr_stddev_converter.equation = matrix32.arr_stddev()

data = arr_stddev_converter.plot(return_df=True)
assert(data[arr_stddev_converter.name][0] == 2.5)
data[arr_stddev_converter.name][0]

2.5

### Array Size
Calculates the size of an array. The size of the highest level will be returned (for example 3 for a 3 by 2 matrix).

Example (size of matrix32 = 3)

In [16]:
arr_size_converter = model.converter("arr_size_converter")
arr_size_converter.equation = matrix32.arr_size()

data = arr_size_converter.plot(return_df=True)
assert(data[arr_size_converter.name][0] == 3)
data[arr_size_converter.name][0]

3

### Array Dot
The Dot function uses matrix/vector multiplication logic. It is possible to input illegal arguments. Legal inputs are:
- Any vector * constant => vector
- Any matrix * constant => matrix
- Vector of size m * Vector of size m => constant
- Vector of size m * Matrix of size mxn => vector of size m
- Matrix of size mxn * Vector of size n => vector of size n
- Matrix of size mxn * Matrix of size nxp => matrix of size mxp

Below are examples for the described dot operations:

#### Vector x Constant:

In [56]:
def get_vector_as_array(element, length):
    data = element.plot(return_df=True)
    arr = []
    for i in range(length):
        arr.append(f"{data[element.name].iloc[i].columns[0]}: {data[element.name].iloc[i].values[0][0]}")
    return arr

vec_dot_constant_converter = model.converter("vec_dot_constant_converter")
vec_dot_constant_converter.equation = vector3.dot(constant)

get_vector_as_array(vec_dot_constant_converter, 3)

['vec_dot_constant_converter[0]: 200.0',
 'vec_dot_constant_converter[1]: 300.0',
 'vec_dot_constant_converter[2]: 400.0']

#### Vector x Vector

In [57]:
vec_dot_vec_converter = model.converter("vec_dot_vec_converter")
vec_dot_vec_converter.equation = vector3.dot(vector3)

data = vec_dot_vec_converter.plot(return_df=True)
data[vec_dot_vec_converter.name][0]

29.0

#### Vector x Matrix

In [58]:
vec_dot_mat_converter = model.converter("vec_dot_mat_converter")
vec_dot_mat_converter.equation = vector3.dot(matrix33)

get_vector_as_array(vec_dot_mat_converter, 3)

['vec_dot_mat_converter[0]: 51.0',
 'vec_dot_mat_converter[1]: 60.0',
 'vec_dot_mat_converter[2]: 69.0']

#### Matrix x Vector

In [59]:
mat_dot_vec_converter = model.converter("mat_dot_vec_converter")
mat_dot_vec_converter.equation = matrix33.dot(vector3)

get_vector_as_array(mat_dot_vec_converter, 3)

['mat_dot_vec_converter[0]: 29.0',
 'mat_dot_vec_converter[1]: 56.0',
 'mat_dot_vec_converter[2]: 83.0']

#### Matrix x Matrix

### Math operations
Mathematical operations are executed on a per-element basis. That means that for all basic mathematical operations both arrays have to have equal size, the exception being array constant operations. Below are examples for all basic mathematical operations.\

#### Multiplication
Element-wise multiplication works with arrays of the same size or arrays and constants.

Example: ([2.0, 3.0, 4.0] * [2.0, 3.0, 4.0] = [4.0, 9.0, 16.0]):

In [61]:
vec_mul_vec_converter = model.converter("vec_mul_vec_converter")
vec_mul_vec_converter.equation = vector3 * vector3

get_vector_as_array(vec_mul_vec_converter, 3)

['vec_mul_vec_converter[0]: 4.0',
 'vec_mul_vec_converter[1]: 9.0',
 'vec_mul_vec_converter[2]: 16.0']

#### Addition
Element-wise multiplication works with arrays of the same size or arrays and constants.

Example: ([2.0, 3.0, 4.0] + [2.0, 3.0, 4.0] = [4.0, 6.0, 8.0]):

In [62]:
vec_add_vec_converter = model.converter("vec_add_vec_converter")
vec_add_vec_converter.equation = vector3 + vector3

get_vector_as_array(vec_add_vec_converter, 3)

['vec_add_vec_converter[0]: 4.0',
 'vec_add_vec_converter[1]: 6.0',
 'vec_add_vec_converter[2]: 8.0']

#### Division
Element-wise division works with arrays of the same size or arrays and constants.

Example: ([2.0, 3.0, 4.0] / [4.0, 3.0, 2.0] = [4.0, 6.0, 8.0]):

In [ ]:
vec_div_vec_converter = model.converter("vec_div_vec_converter")
vector3_alternative = model.converter("converter_vector3_alternative")
vector3_alternative.setup_vector(3, [4.0, 3.0, 2.0])
vec_div_vec_converter.equation = vector3 / vector3_alternative

get_vector_as_array(vec_div_vec_converter, 3)